# 🎯 Draccus 工作原理完整演示

这个 notebook 将一步步展示：
- 机器类型如何传递给 RobotConfig
- 注册表机制如何工作
- `@classmethod` 和 `@property` 的作用
- 完整的命令行参数处理流程


---
## 📦 第1部分：定义注册表基类 (ChoiceRegistry)

这是 draccus 的核心机制：一个注册表系统


In [ ]:
class ChoiceRegistry:
    """注册表基类 - 模拟 draccus.ChoiceRegistry"""
    
    _registry = {}  # 类变量：存储所有注册的子类 {名字: 类}
    
    @classmethod
    def register_subclass(cls, name):
        """装饰器：注册子类到注册表
        
        参数:
            name: 注册的名字，比如 'so101_follower'
        """
        def decorator(subclass):
            print(f"📝 注册：'{name}' -> {subclass.__name__}")
            # 1. 将子类添加到注册表
            cls._registry[name] = subclass
            # 2. 在子类上记住注册的名字
            subclass._choice_name = name
            return subclass
        return decorator
    
    def get_choice_name(self, klass):
        """获取类注册时的名字"""
        return klass._choice_name
    
    @classmethod
    def get_subclass_by_name(cls, name):
        """根据名字查找注册的子类"""
        return cls._registry.get(name)

print("✅ ChoiceRegistry 类定义完成")


### 💡 解释：为什么用 `@classmethod`？

```python
@classmethod
def register_subclass(cls, name):
    ...
```

**因为：**
1. `_registry` 是类级别的变量（所有实例共享）
2. 注册时还不需要创建实例
3. 可以直接用类调用：`RobotConfig.register_subclass('so101')`


---
## 🤖 第2部分：定义机器人配置基类 (RobotConfig)

继承 ChoiceRegistry，增加机器人特有的功能


In [ ]:
class RobotConfig(ChoiceRegistry):
    """机器人配置基类"""
    
    def __init__(self, port, id=None):
        """初始化机器人配置
        
        注意：这里没有 type 参数！
        """
        self.port = port
        self.id = id
        print(f"  → RobotConfig.__init__() 被调用")
        print(f"     参数: port={port}, id={id}")
        print(f"     ⚠️  注意：没有 type 参数！")
    
    @property
    def type(self):
        """动态属性：返回机器类型
        
        每次访问 robot.type 时，会动态查找注册时的名字
        """
        result = self.get_choice_name(self.__class__)
        print(f"  → 访问 .type 属性 -> 动态计算返回: '{result}'")
        return result

print("✅ RobotConfig 类定义完成")


### 💡 解释：为什么用 `@property`？

```python
@property
def type(self):
    return self.get_choice_name(self.__class__)
```

**因为：**
1. `type` 不是存储的变量，而是动态计算的
2. 但我们希望像访问变量一样使用：`robot.type`（不需要括号）
3. 而不是：`robot.get_type()`（需要括号）


---
## 📝 第3部分：注册具体的机器人类型

使用装饰器注册两种机器人配置


In [ ]:
# 注册 SO101 机器人
@RobotConfig.register_subclass("so101_follower")  # ← 注册名字
class SO101FollowerConfig(RobotConfig):
    """SO101 机器人配置"""
    pass


In [ ]:
# 注册 SO100 机器人
@RobotConfig.register_subclass("so100_follower")  # ← 注册名字
class SO100FollowerConfig(RobotConfig):
    """SO100 机器人配置"""
    pass


### 🔍 查看注册表内容

现在我们可以看到注册表中保存了什么：


In [ ]:
print("\n" + "="*60)
print("✅ 注册完成！注册表内容：")
print("="*60)
print(f"RobotConfig._registry = {RobotConfig._registry}")
print("="*60 + "\n")


---
## 🚀 第4部分：模拟 draccus 的工作流程

定义一个函数，模拟 `@draccus.wrap()` 处理命令行参数的过程


In [ ]:
def simulate_draccus(robot_type, robot_port):
    """模拟 draccus 的完整工作流程
    
    参数:
        robot_type: 机器类型，如 'so101_follower'
        robot_port: 端口，如 'COM24'
    """
    
    print(f"\n{'='*60}")
    print(f"🎬 模拟命令行输入：")
    print(f"  --robot.type={robot_type}")
    print(f"  --robot.port={robot_port}")
    print(f"{'='*60}\n")
    
    # 步骤1：解析命令行参数为字典
    print("📋 步骤1: 解析命令行参数")
    params = {
        "type": robot_type,
        "port": robot_port
    }
    print(f"  解析结果: {params}\n")
    
    # 步骤2：查找注册表
    print("🔍 步骤2: 查找注册表")
    print(f"  查找 type='{robot_type}'...")
    ConfigClass = RobotConfig.get_subclass_by_name(robot_type)
    print(f"  ✅ 找到类: {ConfigClass.__name__}\n")
    
    # 步骤3：创建实例（注意：不传 type！）
    print("🏗️  步骤3: 创建实例")
    print(f"  调用: {ConfigClass.__name__}(port='{robot_port}')")
    print(f"  ⚠️  注意：只传 port，不传 type！")
    robot = ConfigClass(port=robot_port)
    print(f"  ✅ 实例创建完成\n")
    
    # 步骤4：访问 type 属性
    print("🎯 步骤4: 访问 type 属性")
    print(f"  调用: robot.type")
    result = robot.type
    print(f"  ✅ 结果: '{result}'\n")
    
    # 步骤5：验证
    print("✔️  步骤5: 验证")
    print(f"  robot.port = '{robot.port}'")
    print(f"  robot.type = '{robot.type}'")
    print(f"  robot.__class__.__name__ = '{robot.__class__.__name__}'")
    print(f"  type(robot) = {type(robot)}")
    
    return robot

print("✅ 模拟函数定义完成")


---
## 🎬 第5部分：运行演示 - 创建 SO101 机器人

模拟命令行：`--robot.type=so101_follower --robot.port=COM24`


In [ ]:
robot1 = simulate_draccus("so101_follower", "COM24")


### 🔬 测试创建的对象


In [ ]:
print("\n" + "🔹"*30)
print("测试 robot1：")
print(f"  类型：{type(robot1)}")
print(f"  robot1.type：{robot1.type}")
print(f"  robot1.port：{robot1.port}")


---
## 🎬 第6部分：运行演示 - 创建 SO100 机器人

模拟命令行：`--robot.type=so100_follower --robot.port=COM42`


In [ ]:
robot2 = simulate_draccus("so100_follower", "COM42")
